# Reducing GPT Primer Files

Welcome! This Python notebook is a step-by-step guide for transforming a GPT Primer file into a reduced version.

### Background

A GPT Primer file, often used with OpenAI's language model, is a YML file that consists of metadata used to influence, constrain, and contextualize ChatGPT responses. This notebook will help you create a reduced version of such files, improving efficiency, reducing token usage, making for more consistent outputs.

## Function to import and primer from a file.

In [23]:
primer_name = 'data_science_tutor'

In [24]:
'''
Load the primer file from a source directory
'''
def get_primer():
    input_fileType = '.yml'

    with open('./primers/' + primer_name + input_fileType, 'r') as f:
        primer = f.read()
        # print(primer)
        return primer

result = get_primer()
result_temp = get_primer()

## Functions to manipulate the primer text

In [25]:
'''
Remove line breaks and replace with '/n' to create a single line of text.
'''
def remove_line_breaks(inc_result = result):
    out_result = inc_result.replace('\n', 'dfd')    
    return out_result

# Test for remove_line_breaks()
# test_result = remove_line_breaks(result_temp)
# print('test_result :', test_result)



test_result : initiation: dfd  - "This is a new conversation. Previous interactions don't influence this one."dfddfdinstructions: dfd  - "You are an AI defined as 'system', with a detailed profile provided. The 'user' is described in their profile. Respond to the 'prompt' section in a way that aligns with the information about both 'system' and 'user'."dfddfdconversation:dfd  - role_01: systemdfd	    profile: dfd	      skills: dfd	        - "Data Science Expertise"dfd	        - "Broad Knowledge of Data Science Tools and Methodologies"dfd	        - "Patient and Thorough Guidance"dfd	        - "Simplification of Complex Concepts"dfd	        - "Analogical Explanation"dfd	        - "Prompt Engineering"dfd	        - "Architectural Planning"dfd	        - "Technical Writing"dfd	        - "Art Direction"dfd	        - "Statistical Analysis"dfd	        - "Professional Coaching"dfd	        - "Professional Tutoring"dfd	      goals: dfd	        - "Guiding, mentoring, and providing best-practice adv

In [26]:
'''
Remove all spaces from result.
'''
def remove_spaces():
    out_result = result.replace(' ', '')
    # print('remove_spaces() :',out_result)
    
    return out_result

test_result = remove_line_breaks(result_temp)
print('test_result :', test_result)

# Function to write results to file aof the same name

In [27]:
'''
Write result into a file with the extension type .ppo at the file path ./reduced_primer/<primer_name>.ppo
'''
def write_reduced_primer():
    output_fileType = '.ppo'
    with open('./reduced_primers/' + primer_name + output_fileType, 'w') as f:
        f.write(result)
        # print('write_reduced_primer() :', inc_result)
    
    

In [28]:
result = remove_line_breaks()
# remove_spaces()
write_reduced_primer()




## Save string text to clipboard